In [1]:
%matplotlib inline

%matplotlib inline
%load_ext autoreload
%autoreload 2


import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cftime
import dask
import xarrayutils
import cartopy.crs as ccrs
from xmip.preprocessing import combined_preprocessing
from xmip.preprocessing import replace_x_y_nominal_lat_lon
from xmip.drift_removal import replace_time
from xmip.postprocessing import concat_experiments
import xmip.drift_removal as xm_dr
import xmip as xm
import xesmf as xe
import datetime
from dateutil.relativedelta import relativedelta

import utils


In [2]:
dask.config.set(**{'array.slicing.split_large_chunks': True})

## Functions

In [3]:
########## WEIGHT YOUR MEAN BY MONTH LENGTH ########## 

In [4]:
#### function to find area of a grid cell from lat/lon ####
def find_area(ds, R = 6378.1):
    """ ds is the dataset, i is the number of longitudes to assess, j is the number of latitudes, and R is the radius of the earth in km. 
    Must have the ds['lat'] in descending order (90...-90)
    Returns Area of Grid cell in km"""
    circumference = (2*np.pi)*R
    deg_to_m = (circumference/360) 
    dy = (ds['lat_b'].roll({'lat_b':-1}, roll_coords = False) - ds['lat_b'])[:-1]*deg_to_m

    dx1 = (ds['lon_b'].roll({'lon_b':-1}, roll_coords = False) - 
           ds['lon_b'])[:-1]*deg_to_m*np.cos(np.deg2rad(ds['lat_b']))
    
    dx2 = (ds['lon_b'].roll({'lon_b':-1}, roll_coords = False) - 
           ds['lon_b'])[:-1]*deg_to_m*np.cos(np.deg2rad(ds['lat_b'].roll({'lat_b':-1}, roll_coords = False)[:-1]))
    
    A = .5*(dx1+dx2)*dy
    
    #### assign new lat and lon coords based on the center of the grid box instead of edges ####
    A = A.assign_coords(lon_b = ds.lon.values,
                    lat_b = ds.lat.values)
    A = A.rename({'lon_b':'lon','lat_b':'lat'})

    A = A.transpose()
    
    return(A)

In [5]:
# NEED TO REDOWNLOAD, DATA INCOMPLETE
# ds_control, ds_pulse, G = import_regrid_calc('cmip6_data/tas_Amon_CanESM5_esm-piControl_r1i1p2f1*', 
#                                              'cmip6_data/tas_Amon_CanESM5_esm-pi-CO2pulse_r1i1p2f1*', 
#                                              ds_out, pulse_size = 100)

In [6]:
#### NEED TO FIGURE OUT IF F NEEDS TO BE THE SAME IN ALL RUNS TO COMPARE ACROSS!!! ########

In [7]:
### NEED TO FIX ACCESS DATA ###

In [8]:
### NEED TO DO SOME SORT OF MATCH UP OF INTERANNUAL VARIABILITY--  #####
### EG: THEY MAY BE STARTING IN DIFFERENT YEARS, LEADING TO DIFFERENCES IN THE MEAN THAT ARE ARTIFACTS ######

## Data

Data for this is from https://gmd.copernicus.org/articles/11/1133/2018/ CDRMIP data, where pi-CO2pulse is the 100GtC pulse and piControl is the control

In [9]:
model_run_control_dict = {
                         'CANESM5_p2':'CanESM5_esm-piControl_r1i1p2f1*',
                          #'CANESM5_p1':'CanESM5_esm-piControl_r1i1p1f1*',
                         }

model_run_pulse_dict = {
                       'CANESM5_r1':'CanESM5_esm-pi-CO2pulse_r1i1p2f1*',
                       'CANESM5_r2':'CanESM5_esm-pi-CO2pulse_r2i1p2f1*',
                       'CANESM5_r3':'CanESM5_esm-pi-CO2pulse_r3i1p2f1*'}



In [10]:
#define our output grid size

ds_out = xr.Dataset(
    {
        "lat": (["lat"], np.arange(-89.5, 90.5, 1.0)),
        "lon": (["lon"], np.arange(0, 360, 1)),
        "lat_b": (["lat_b"], np.arange(-90.,91.,1.0)),
        "lon_b":(["lon_b"], np.arange(.5, 361.5, 1.0))
    }
)

In [11]:
A = find_area(ds_out)

In [12]:
def _regrid_cont_pulse(ds_control, ds_pulse, ds_out):
    regridder = xe.Regridder(ds_control, ds_out, "bilinear")
    attrs = ds_control.attrs
    ds_control = regridder(ds_control) 
    ds_control.attrs = attrs
    
    regridder = xe.Regridder(ds_pulse, ds_out, "bilinear")
    attrs = ds_pulse.attrs
    ds_pulse = regridder(ds_pulse) 
    ds_pulse.attrs = attrs
    
    return(ds_control, ds_pulse)


def _calc_greens(ds_control, ds_pulse, variable, pulse_size = 100):
    
    #A = find_area(ds_control.isel(time = 0), lat_bound_nm = 'lat_bounds', lon_bound_nm = 'lon_bounds')
    G = (ds_pulse[variable] - ds_control[variable])/(pulse_size)
    G = G.groupby('time.year').mean()
    G.attrs = ds_pulse.attrs
    

In [13]:
ds_control = {}
ds_pulse = {}
G = {}

anom_control = {}
anom_pulse = {}
anom_G = {}
for m1 in model_run_pulse_dict.keys():
    m2 = 'CANESM5_p2'
    ds_control[m1] = xr.open_mfdataset(f'cmip6_data/tas_Amon_{model_run_control_dict[m2]}', use_cftime=True)
    ds_pulse[m1] = xr.open_mfdataset(f'cmip6_data/tas_Amon_{model_run_pulse_dict[m1]}', use_cftime=True)
    
    ds_pulse[m1]['time'] = ds_control[m1]['time'][:len(ds_pulse[m1]['time'])]
    ds_control[m1] = replace_x_y_nominal_lat_lon(ds_control[m1])
    ds_pulse[m1] = replace_x_y_nominal_lat_lon(ds_pulse[m1])

    ds_control[m1], ds_pulse[m1] = utils._regrid_cont_pulse(ds_control[m1], ds_pulse[m1], ds_out)
    G[m1] = utils._calc_greens(ds_control[m1], ds_pulse[m1], 'tas', pulse_size = 100)

/home/emfreese/anaconda3/envs/gchp/lib/python3.9/site-packages/xmip/preprocessing.py:209: UserWarning: No x and y found in dimensions for source_id:CanESM5. This likely means that you forgot to rename the dataset or this is the German unstructured model
  warnings.warn(
/home/emfreese/anaconda3/envs/gchp/lib/python3.9/site-packages/xesmf/frontend.py:555: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(
/home/emfreese/anaconda3/envs/gchp/lib/python3.9/site-packages/xesmf/frontend.py:555: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(
/home/emfreese/anaconda3/envs/gchp/lib/python3.9/site-packages/xmip/preprocessing.py:209: UserWarning: No x and y found in dimensions for source_id:CanESM5. This likely means that you forgot to rename the 

In [21]:
G['CANESM5_r1']

<xarray.DataArray 'tas' (year: 200, lat: 180, lon: 360)>
dask.array<stack, shape=(200, 180, 360), dtype=float64, chunksize=(1, 180, 360), chunktype=numpy.ndarray>
Coordinates:
    height   float64 2.0
  * lon      (lon) int64 0 1 2 3 4 5 6 7 8 ... 352 353 354 355 356 357 358 359
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
  * year     (year) datetime64[ns] 2000-01-01 2001-01-01 ... 2199-01-01
Attributes:
    CCCma_model_hash:            62e222548da1e85b9485b625c4aa21c138062cf1
    CCCma_parent_runid:          p2-pifree
    CCCma_pycmor_hash:           26c970628162d607fffd14254956ebc6dd3b6f49
    CCCma_runid:                 p2-pifree-add100-01
    Conventions:                 CF-1.7 CMIP-6.2
    YMDH_branch_time_in_child:   5401:01:01:00
    YMDH_branch_time_in_parent:  5401:01:01:00
    activity_id:                 CDRMIP
    branch_method:               Spin-up documentation
    branch_time_in_child:        1296115.0
    branch_time_in_parent:       1296115.0
    contact:                     ec.cccma.info-info.ccmac.ec@canada.ca
    creation_date:               2019-08-20T20:38:28Z
    data_specs_version:          01.00.30
    experiment:                  pulse addition of 100 Gt carbon to pre-indus...
    experiment_id:               esm-pi-CO2pulse
    external_variables:          areacella
    forcing_index:               1
    frequency:                   mon
    further_info_url:            https://furtherinfo.es-doc.org/CMIP6.CCCma.C...
    grid:                        T63L49 native atmosphere, T63 Linear Gaussia...
    grid_label:                  gn
    history:                     2019-08-20T20:38:28Z ;rewrote data to be con...
    initialization_index:        1
    institution:                 Canadian Centre for Climate Modelling and An...
    institution_id:              CCCma
    mip_era:                     CMIP6
    nominal_resolution:          500 km
    parent_activity_id:          CMIP
    parent_experiment_id:        esm-piControl
    parent_mip_era:              CMIP6
    parent_source_id:            CanESM5
    parent_time_units:           days since 1850-01-01 0:0:0.0
    parent_variant_label:        r1i1p1f1
    physics_index:               2
    product:                     model-output
    realization_index:           1
    realm:                       atmos
    references:                  Geophysical Model Development Special issue ...
    source:                      CanESM5 (2019): \naerosol: interactive\natmo...
    source_id:                   CanESM5
    source_type:                 AOGCM
    sub_experiment:              none
    sub_experiment_id:           none
    table_id:                    Amon
    table_info:                  Creation Date:(09 May 2019) MD5:38ea1fbdf3f3...
    title:                       CanESM5 output prepared for CMIP6
    tracking_id:                 hdl:21.14100/d890df5f-7780-483f-9800-0f21971...
    variable_id:                 tas
    variant_label:               r1i1p2f1
    version:                     v20190429
    license:                     CMIP6 model data produced by The Government ...
    cmor_version:                3.5.0

In [14]:
base = datetime.datetime.strptime("2000", "%Y")
for m in G.keys():
    G[m]['year'] = [base + relativedelta(years=x) for x in range(len(G[m]['year']))]
    #anom_G[m]['year'] = [base + relativedelta(years=x) for x in range(len(anom_G[m]['year']))]

In [40]:
G_ds = xr.concat([G[m] for m in G.keys()], pd.Index([m for m in G.keys()], name='model'))
#anom_G_ds = xr.concat([anom_G[m] for m in anom_G.keys()], pd.Index([m for m in anom_G.keys()], name='model'))

In [41]:
G_mean_ds = G_ds.mean(dim = 'model')
#anom_G_mean_ds = anom_G_ds.mean(dim = 'model')

## Save Green's Functions

In [42]:
G_ds.to_netcdf('Outputs/G_canesm5_ds.nc4')
G_mean_ds.to_netcdf('Outputs/G_mean_canesm5_ds.nc4')

# anom_G_ds.to_netcdf('Outputs/anom_G_ds.nc4')
# anom_G_mean_ds.to_netcdf('Outputs/anom_G_mean_ds.nc4')